In [32]:
import os
import re
import pandas as pd
import xml.etree.ElementTree as ET

def search_target_xml(rootDirFullPath, regex_pattern):
  match_files = []
  for root, _, files in os.walk(rootDirFullPath):
    for filename in files:
       if re.match(regex_pattern, filename):
          match_files.append(os.path.join(root, filename))
  return match_files
   
def try_get_xml_element_text(xml_root, element_full_path):
   element = xml_root.find(element_full_path)
   if element is None:
      return "none"
   return element.text

def get_module_descritpion_dataFrame(targetXmlFiles):
  outputData = [["ModuleName", "JPN", "ENG", "FileName"]]
  for xml in targetXmlFiles:
    xml_tree = ET.parse(xml)
    xml_root = xml_tree.getroot()

    product_name = try_get_xml_element_text(xml_root,'./AdapterInfo/ProdName')
    descTextModuleJpn = try_get_xml_element_text(xml_root,'./AdapterInfo/DescTextMulLan/JPN')
    descTextModuleEng = try_get_xml_element_text(xml_root,'./AdapterInfo/DescTextMulLan/ENG')

    data = [product_name, descTextModuleJpn, descTextModuleEng, os.path.basename(xml)]
    outputData.append(data)
  df = pd.DataFrame(outputData)
  return df

In [36]:
def get_param_dataFrame(xmlFiles):
  languages = ['JPN', 'ENG']
  header = []
  for l in languages:
    header.append(f'Comment({l})')
    header.append(f'Description({l})')
    header.append(f'Values({l})')
  header.append('Src file')
  outputData = [header]
  
  for xml in xmlFiles:
    xml_tree = ET.parse(xml)
    params = xml_tree.findall('.//ParameterTable/Parameter')

    for p in params:
      rowData = []
      for l in languages:
        commnet = try_get_xml_element_text(p, f'ParameterNameMulLan/{l}')
        description = try_get_xml_element_text(p, f'HelpStringMulLan/{l}')
        values = try_get_xml_element_text(p, f'EnumMulLan/{l}')
        rowData.append(commnet)
        rowData.append(description)
        rowData.append(values)
      rowData.append(os.path.basename(xml))
      outputData.append(rowData)
  print(outputData)
  df = pd.DataFrame(outputData)
  return df


In [37]:
def main():
  dirFullPath = input()
  targetPattern = r"367_43_2(\d{2})_1_1\.xml"
  targetXmls = search_target_xml(dirFullPath, targetPattern)

  df = get_param_dataFrame(targetXmls)
  df.to_csv(os.path.join('C:\work', 'output.csv'), encoding = 'utf-8')# shift-jisだと変換不可の文字がいたので、いったんutf-8で保存

main()

[['Comment(JPN)', 'Description(JPN)', 'Values(JPN)', 'Comment(ENG)', 'Description(ENG)', 'Values(ENG)', 'Src file'], ['MACアドレス', None, 'none', 'MAC address', None, 'none', '367_43_200_1_1.xml'], ['ステータス', '本機のステータスを格納します。', '0,"IP アドレス重複",1,"ロータリースイッチ不正",2,"予約",3,"予約",4,"予約",5,"予約",6,"予約",7,"予約",8,"拡張バス通信異常",9,"ユニット台数不正",10,"未対応ユニット",11,"予約",12,"予約",13,"予約",14,"予約",15,"予約"', 'none', 'none', 'none', '367_43_200_1_1.xml'], ['入力時定数', '入力時定数を設定します。', '0,"10ms",1,"1ms",2,"25μs",3,"300μs"', 'Input time constant', 'Set the input time constant.', '0,"10ms",1,"1ms",2,"25us",3,"300us"', '367_43_210_1_1.xml'], ['ID番号占有数', 'ID番号占有数を格納します。', None, 'Occupied ID number', 'Store the occupied number of the ID number.', None, '367_43_210_1_1.xml'], ['プロダクトコード', 'プロダクトコードを格納します', None, 'Product code', 'Store the product code.', None, '367_43_210_1_1.xml'], ['リビジョン', 'リビジョンを格納します。上位バイトがメジャーリビジョンを、下位バイトがマイナーリビジョンを格納します。', None, 'Revision', 'Store the revision. The high-order byte stores the major revision 